In [1]:
import os
import numpy as np

base_path = '/mnt/disk1/heonseok/MPMLD/output'

In [82]:
dataset = 'location'

z_dim_list = [
    # '16',
    '64',
]

setsize_list = [
    '2000',
]

lr_list = [
    '0.001',
    # '0.01',
    # '0.1',
]

ref_list = [
    '0.1',
]

arc_list = [
    # 'A',
    # 'B',
    'C',
]

dis_type_list = [
    'type5',
]

cw_list = [
    '0.01',
    '0.1',
    '1.0',
]

mw_list = [
    '0.01',
    '0.1',
    '1.0',
]

recon_model_list = []
for z_dim in z_dim_list:
    for setsize in setsize_list:
        for lr in lr_list:
            for ref in ref_list:
                for arc in arc_list:
                    for dis_type in dis_type_list:
                        for cw in cw_list:
                            for mw in mw_list:
                                recon_model = 'AE_z{}_setsize{}_lr{}_ref{}_arc{}_{}_cw{}_mw{}'.format(
                                    z_dim, setsize, lr, ref, arc, dis_type, cw, mw,
                                )
                                recon_model_list.append(recon_model)

for recon_model in recon_model_list:
    print('\''+recon_model+'\',')


'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0',


In [83]:
for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    class_acc_full_list = []
    class_acc_content_list = []
    class_acc_style_list = []
    membership_acc_full_list = []
    membership_acc_content_list = []
    membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_acc_path = os.path.join(recon_repeat_path, 'acc.npy')
            recon_acc = np.load(recon_acc_path, allow_pickle=True).item()

            class_acc_full_list.append(recon_acc['class_acc_full'])
            class_acc_content_list.append(recon_acc['class_acc_content'])
            class_acc_style_list.append(recon_acc['class_acc_style'])
            membership_acc_full_list.append(recon_acc['membership_acc_full'])
            membership_acc_content_list.append(recon_acc['membership_acc_content'])
            membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    result_list = [
        *recon_model.split('_'),
        np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
        np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    ]

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.01,0.8648,0.7384,0.1299,0.9468,0.8252,0.8891
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.1,0.8575,0.7211,0.1383,0.8347,0.5001,0.8200
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw1.0,0.8519,0.7232,0.1353,0.6627,0.5000,0.8039
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw0.01,0.8434,0.7628,0.0671,0.9149,0.8406,0.8485
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw0.1,0.7864,0.7111,0.0647,0.6251,0.5161,0.7657
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw1.0,0.8397,0.7250,0.0676,0.5000,0.5000,0.7719
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw0.01,0.9057,0.7213,0.0655,0.9254,0.8527,0.6749
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw0.1,0.8589,0.6660,0.0655,0.7097,0.5000,0.6473
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw1.0,0.8531,0.7120,0.0655,0.4999,0.5000,0.6651


In [77]:
for recon_model in recon_model_list:
    class_path = os.path.join(base_path, dataset, 'classifier', recon_model+'_FCNClassifierA')

    result_list = []
    for recon_type in ['full_z', 'content_z', 'style_z']:
        train_acc_list = []
        valid_acc_list = []
        test_acc_list = []
        for repeat in range(5):
            try:
                class_repeat_path = os.path.join(class_path, recon_type, 'repeat{}'.format(repeat))
                class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
                train_acc_list.append(class_acc['train'])
                valid_acc_list.append(class_acc['valid'])
                test_acc_list.append(class_acc['test'])

            except FileNotFoundError:
                # print('File not found: ', class_repeat_path)
                continue

        result_list.extend([np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))

0.6500,0.5980,0.5770,0.5020,0.3290,0.2855,0.0660,0.0735,0.0590
0.6691,0.5785,0.5665,0.5332,0.2615,0.2130,0.0655,0.0710,0.0560
0.6603,0.5735,0.5550,0.5602,0.4070,0.3640,0.0651,0.0730,0.0510
0.6993,0.6135,0.5835,0.5506,0.4145,0.3705,0.0651,0.0740,0.0545
0.6815,0.5960,0.5645,0.5181,0.3125,0.2835,0.0655,0.0770,0.0540
0.6494,0.5715,0.5350,0.5924,0.4785,0.4465,0.0676,0.0810,0.0540
0.6941,0.6200,0.5845,0.6083,0.4705,0.4275,0.0586,0.0690,0.0615
0.6888,0.6035,0.5760,0.5965,0.4905,0.4430,0.0625,0.0740,0.0530
0.6748,0.5830,0.5465,0.5731,0.4420,0.3935,0.0659,0.0795,0.0530
0.3622,0.3980,0.3635,0.3613,0.3860,0.3505,0.0624,0.0730,0.0530
0.5369,0.5180,0.4900,0.5345,0.5135,0.4880,0.0639,0.0725,0.0520
0.4310,0.4480,0.3950,0.4362,0.4430,0.3945,0.0623,0.0740,0.0535
0.4855,0.4990,0.4710,0.4942,0.4950,0.4740,0.0655,0.0750,0.0550
0.4976,0.4700,0.4350,0.5005,0.4635,0.4385,0.0655,0.0735,0.0545
0.4156,0.3980,0.3670,0.4217,0.3885,0.3570,0.0655,0.0725,0.0525
0.5283,0.5145,0.4845,0.5327,0.5175,0.4800,0.0596,0.0760

In [40]:
classifier_with_raw_data = 'original_setsize2000_FCNClassifierA'
class_path = os.path.join(base_path, dataset, 'classifier', classifier_with_raw_data)
train_acc_list = []
valid_acc_list = []
test_acc_list = []
for repeat in range(5):
    try:
        class_repeat_path = os.path.join(class_path, 'repeat{}'.format(repeat))
        class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
        train_acc_list.append(class_acc['train'])
        valid_acc_list.append(class_acc['valid'])
        test_acc_list.append(class_acc['test'])

    except FileNotFoundError:
        print('File not found: ', class_repeat_path)
        continue

print(np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list))



0.942 0.685 0.6535
